In [2]:
import nltk
import math
import pickle
import re
import string
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()

nltk_words = set(nltk.corpus.stopwords.words('english'))
stop_words = []
for word in nltk_words:
    stop_words.append(word.translate(str.maketrans('', '', string.punctuation)))

def preprocess(sentence):
    res = sentence.lower()
    res = res.translate(str.maketrans('', '', string.punctuation))
    tokenized_words = nltk.word_tokenize(res)
    res = [word for word in tokenized_words if word not in stop_words]
    res = [lemmatizer.lemmatize(r) for r in res]
    res = [re.sub(r"[^A-Za-z]+", '', r) for r in res]
    res = [r for r in res if len(r) > 3]
    return ' '.join(res)

def generate_tficf(reviews, n_cluster = 4):
    tf_word_freq = {}
    for n in range(n_cluster):
        tf_word_freq[n] = {}
   
    tf = []
    for index, review in enumerate(reviews):
        for sentence in nltk.sent_tokenize(review):
            for word in nltk.word_tokenize(sentence):
                if word in tf_word_freq[index]:
                    tf_word_freq[index][word] += 1
                else:
                    tf_word_freq[index][word] = 1

                if word not in tf:
                    tf.append(word)
                                    
    tf_icf = []
    for term in tf:
        tf_in_c = {}
        c_i = 0

        for n in range(n_cluster):
            tf_in_c[n] = 0
            
            if term in tf_word_freq[n]:
                tf_in_c[n] = tf_word_freq[n][term]
                c_i += 1

        icf = 1 + math.log(n_cluster/c_i)
        t = ()
        t += (term,)
        for n in range(n_cluster):
            t += (tf_in_c[n]*icf, )
        tf_icf.append(t)
    
    return tf_icf

def open_file(kategori):
    f = open('halaman_wikipedia/' + kategori + '-sentence.txt', 'r', encoding='utf-8')
    all_sentences = []
    for line in f:
        all_sentences.append(preprocess(str(line).replace('\n','')))
    return ' '.join(all_sentences)

def save_daftar_halaman(name, daftar):
    pickle_out = open("keyword_aspek/daftar_halaman_" + name + ".pickle","wb")
    pickle.dump(daftar, pickle_out)
    pickle_out.close()

def load_daftar(name):
    pickle_in = open("keyword_aspek/daftar_halaman_" + name + ".pickle", "rb")
    return pickle.load(pickle_in)

def gabung_file(name):
    daftar = load_daftar(name)
    res = []
    for judul in daftar:
        res.append(open_file(judul))
    return ' '.join(res)

def get_keyword(index, top_n, data_tf):
    sorted_word = sorted(data_tf, key=lambda value: value[index], reverse=True)
    keyword = sorted_word[0:top_n]
    max_k = keyword[0][index]
    min_k = keyword[top_n - 1][index]
    keyword = list(map(lambda value: (value[0], (value[1] - min_k) / (max_k - min_k)), keyword))
    return keyword

def run(name, index, top_n, data_tf):
    all_keyword = []
    for a, b in get_keyword(index, top_n, data_tf):
        all_keyword.append(a)
    pickle_out = open("keyword_aspek/" + name + ".pickle","wb")
    pickle.dump(all_keyword, pickle_out)
    pickle_out.close()

data = [('ambience', 1), ('food', 2), ('service', 3), ('price', 4)]

save_daftar_halaman('ambience', ['Theme restaurant', 'Atmosphere (architecture and spatial design)', 'Ambience (sound recording)'])
save_daftar_halaman('food', ['Food', 'Drink', 'Meal'])
save_daftar_halaman('service', ['Customer service', "Waiting staff"])
save_daftar_halaman('price', ["Price", "Pricing"])
    
ambience = gabung_file('ambience')
food = gabung_file('food')
service = gabung_file('service')
price = gabung_file('price')

data_tf = generate_tficf([ambience, food, service, price], 4)
for name, index in data:
    run(name, index, 10, data_tf)